In [1]:
%cd C:/Users/Mathieu/Desktop/Projets/Benter

C:\Users\Mathieu\Desktop\Projets\Benter


In [16]:
import numpy as np
from tqdm import tqdm

from utils import import_data
from constants import PMU_BETTINGS
from winning_horse_models.logistic_regression import LogisticRegressionModel

initial_capital =10000
source="PMU"
code_pari="E_SIMPLE_GAGNANT"
winning_model = LogisticRegressionModel.load_model(trainable=False)

C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\l

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
track_take = [betting for betting in PMU_BETTINGS if betting.name == code_pari][0][
    1
]
np.random.seed(42)
capital_value = initial_capital
records = []
n_races = import_data.get_n_races(
    source=source, on_split="val", remove_nan_previous_stakes=True
)
for x_race, y_race, race_df in tqdm(
    import_data.get_dataset_races(
        source=source,
        on_split="val",
        y_format="rank",
        x_format="sequential_per_horse",
        remove_nan_previous_stakes=True,
    ),
    leave=False,
    total=n_races,
):
    break

  0%|                                                                                                                                                                                                        | 0/11929 [00:00<?, ?it/s]C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Mathieu\Desktop\Projets\Benter\utils\preprocess.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ) / standard_

In [25]:

from utils.expected_return import get_race_expected_return
previous_stakes=race_df['totalEnjeu']
from constants import PMU_MINIMUM_BET_SIZE
capital_fraction=1.0

In [35]:
_minimum_winning_probabilities = 0.0
_expected_return_threshold=150

In [39]:
n_horses = x_race.shape[0]

y_hat_race = winning_model.predict(x=np.expand_dims(x_race, axis=0))[0, :]
y_hat_race = np.where(
    y_hat_race > _minimum_winning_probabilities, y_hat_race, np.zeros((n_horses,))
)

expected_return_race = get_race_expected_return(
    y_hat_race=y_hat_race,
    track_take=track_take,
    previous_stakes=previous_stakes,
    race_bet=PMU_MINIMUM_BET_SIZE * np.ones((n_horses,)),
)/PMU_MINIMUM_BET_SIZE/n_horses
print(expected_return_race)
max_expected_return = expected_return_race.max()

if max_expected_return <= _expected_return_threshold:
    print(np.zeros_like(y_hat_race))

betting = expected_return_race == max_expected_return
assert betting.sum()
betting = betting / betting.sum()
betting = betting * capital_fraction
print(betting)

1005238    0.040968
1005239   -0.030264
1005240   -0.027660
1005241   -0.047795
1005242    0.093418
1005243    0.093115
1005244    0.005635
Name: totalEnjeu, dtype: float64
[0. 0. 0. 0. 0. 0. 0.]
1005238    0.0
1005239    0.0
1005240    0.0
1005241    0.0
1005242    1.0
1005243    0.0
1005244    0.0
Name: totalEnjeu, dtype: float64


In [40]:
expected_return_race.max()


0.09341775762463145

In [41]:
expected_return_race

1005238    0.040968
1005239   -0.030264
1005240   -0.027660
1005241   -0.047795
1005242    0.093418
1005243    0.093115
1005244    0.005635
Name: totalEnjeu, dtype: float64

In [46]:
previous_stakes.sum()

131850.0

In [48]:
PMU_MINIMUM_BET_SIZE*n_horses

1050

In [52]:
race_bet =np.ones((n_horses,))*PMU_MINIMUM_BET_SIZE

In [62]:
odds=(
        race_bet
        / (previous_stakes + race_bet)
        * (1 - track_take)
        * (previous_stakes.sum())
    )

In [63]:
y_hat_race*odds

1005238    188.040280
1005239    105.904467
1005240    115.170429
1005241     66.732460
1005242    188.006495
1005243    216.654535
1005244    145.109936
Name: totalEnjeu, dtype: float64

In [64]:
-race_bet*(1-y_hat_race)

array([-146.52154092, -138.52543831, -145.13033666, -117.4487561 ,
        -91.41505659, -120.60961872, -140.3492555 ])

In [65]:
y_hat_race*odds-race_bet*(1-y_hat_race)

1005238    41.518739
1005239   -32.620971
1005240   -29.959907
1005241   -50.716296
1005242    96.591438
1005243    96.044917
1005244     4.760681
Name: totalEnjeu, dtype: float64

In [68]:
(y_hat_race*odds-race_bet*(1-y_hat_race))/PMU_MINIMUM_BET_SIZE

1005238    0.276792
1005239   -0.217473
1005240   -0.199733
1005241   -0.338109
1005242    0.643943
1005243    0.640299
1005244    0.031738
Name: totalEnjeu, dtype: float64

In [60]:
get_race_expected_return(
    y_hat_race=y_hat_race,
    track_take=track_take,
    previous_stakes=previous_stakes,
    race_bet=PMU_MINIMUM_BET_SIZE * np.ones((n_horses,)),
)

1005238    43.016216
1005239   -31.777591
1005240   -29.042737
1005241   -50.184866
1005242    98.088646
1005243    97.770266
1005244     5.916278
Name: totalEnjeu, dtype: float64

In [67]:
get_race_expected_return(
    y_hat_race=y_hat_race,
    track_take=track_take,
    previous_stakes=previous_stakes,
    race_bet=PMU_MINIMUM_BET_SIZE * np.ones((n_horses,)),
)/PMU_MINIMUM_BET_SIZE

1005238    0.286775
1005239   -0.211851
1005240   -0.193618
1005241   -0.334566
1005242    0.653924
1005243    0.651802
1005244    0.039442
Name: totalEnjeu, dtype: float64